In [ ]:
! pip install -Uq transformers sentence_transformers datasets fsspec faiss-gpu-cu12

In [ ]:
import wandb
wandb.init(mode="disabled")

### **Load Dataset**

In [ ]:
from datasets import load_dataset

am_relevance_dataset = load_dataset("yosefw/amharic-news-retrieval-dataset-v2-with-negatives-V2")
am_relevance_dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['query_id', 'passage_id', 'query', 'passage', 'category', 'link', 'source_dataset', 'negative_passages'],
        num_rows: 61469
    })
    test: Dataset({
        features: ['query_id', 'passage_id', 'query', 'passage', 'category', 'link', 'source_dataset', 'negative_passages'],
        num_rows: 6832
    })
})

In [ ]:
test_passage_ids = set(am_relevance_dataset["test"]["passage_id"])
len(test_passage_ids)

6764

In [ ]:
# Binary dataset

from datasets import Dataset
from tqdm import tqdm
import random

ds_rows = []
for row in tqdm(am_relevance_dataset["train"]):
  ds_rows.append({
      "query_id": row["query_id"],
      "passage_id": row["passage_id"],
      "query": row["query"],
      "passage": row["passage"],
      "label": 1
    })

  neg_passages = row["negative_passages"]

  # Remove negative passages that exist in the test set
  # neg_passages = list(filter(lambda x: x["passage_id"] not in test_passage_ids, neg_passages))

  for neg in neg_passages[:3]+neg_passages[-4:]:
    ds_rows.append({
      "query_id": row["query_id"],
      "passage_id": neg["passage_id"],
      "query": row["query"],
      "passage": neg["passage"],
      "label": 0
    })

# ds_rows = ds_rows[:64000] #
relevance_train_dataset = Dataset.from_list(ds_rows).shuffle(seed=42)
relevance_train_dataset

In [ ]:
relevance_test_dataset = am_relevance_dataset["test"]#.select(range(100))
relevance_test_dataset

Dataset({
    features: ['query_id', 'passage_id', 'query', 'passage', 'category', 'link', 'source_dataset', 'negative_passages'],
    num_rows: 6832
})

### **Model Training**

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder, CrossEncoderModelCardData
from sentence_transformers.cross_encoder.evaluation import CrossEncoderRerankingEvaluator

# First, we define the transformer model we want to fine-tune
model_name = "rasyosef/roberta-medium-amharic"

# We set num_labels=1, which predicts a continuous score between 0 and 1
model = CrossEncoder(
    model_name,
    num_labels=1,
    max_length=510,
    device="cuda",
    model_card_data=CrossEncoderModelCardData(
        language="am",
        license="mit",
        model_name="roberta-amharic-reranker-medium",
    ),
  )

In [ ]:
from sentence_transformers.util import mine_hard_negatives
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
embedding_model = SentenceTransformer(
    "rasyosef/roberta-amharic-text-embedding-base",
    device="cuda",
  )

hard_eval_dataset = mine_hard_negatives(
        relevance_test_dataset.select_columns(["query", "passage"]),
        embedding_model,
        corpus=am_relevance_dataset["train"]["passage"], #[:8000],  # Use the full dataset as the corpus
        num_negatives=50,  # How many documents to rerank
        batch_size=128,
        include_positives=False,
        output_format="n-tuple",
        use_faiss=True,
    )

print(hard_eval_dataset)

In [ ]:
row = hard_eval_dataset[0]
for col in hard_eval_dataset.column_names:
  print(col,": ", row[col][:64])

In [ ]:
# We add an evaluator, which evaluates the performance during training]
dev_evaluator = CrossEncoderRerankingEvaluator(
  samples=[
    {
        "query": row["query"],
        "positive": [row["passage"]],
        "negative": [row[column_name] for column_name in hard_eval_dataset.column_names[2:]],
    }
    for row in hard_eval_dataset.shuffle(seed=42).select(range(1600))
  ],
  at_k=10,
  batch_size=128,
  name="amh-passage-retrieval-dev",
  always_rerank_positives=True,
  show_progress_bar=True
)

In [ ]:
dev_evaluator(model)

{'amh-passage-retrieval-dev_map': 0.08924252914778016,
 'amh-passage-retrieval-dev_mrr@10': 0.05746974206349206,
 'amh-passage-retrieval-dev_ndcg@10': 0.08978664796439237}

#### **Train**

In [ ]:
import torch
from sentence_transformers.cross_encoder import CrossEncoderTrainer, CrossEncoderTrainingArguments
from sentence_transformers.cross_encoder.losses import BinaryCrossEntropyLoss
from sentence_transformers.training_args import BatchSamplers

run_name = f"roberta-amharic-reranker-medium"

num_train_epochs = 4
train_batch_size = 64

args = CrossEncoderTrainingArguments(
    # Required parameter:
    output_dir=f"models/{run_name}",
    # Optional training parameters:
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=train_batch_size,
    learning_rate=4e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    # weight_decay=0.1,
    fp16=True,
    batch_sampler=BatchSamplers.NO_DUPLICATES,
    dataloader_num_workers=2,
    load_best_model_at_end=True,
    metric_for_best_model="amh-passage-retrieval-dev_mrr@10",
    # Optional tracking/debugging parameters:
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=2,
    # logging_first_step=True,
    run_name=run_name,  # Will be used in W&B if `wandb` is installed
    seed=42,
)

loss = BinaryCrossEntropyLoss(model=model, pos_weight=torch.tensor(7))

# 6. Create the trainer & start training
trainer = CrossEncoderTrainer(
    model=model,
    args=args,
    train_dataset=relevance_train_dataset.select_columns(['query', 'passage', 'label']),
    loss=loss,
    evaluator=dev_evaluator,
)

In [ ]:
trainer.train()

In [ ]:
# Save latest model
model.save_pretrained(run_name + "-latest")

# **Evaluation**

In [ ]:
# We add an evaluator, which evaluates the performance during training]
evaluator = CrossEncoderRerankingEvaluator(
  samples=[
    {
        "query": row["query"],
        "positive": [row["passage"]],
        "negative": [row[column_name] for column_name in hard_eval_dataset.column_names[2:]],
    }
    for row in hard_eval_dataset
  ],
  at_k=10,
  batch_size=128,
  name="amh-passage-retrieval-dev",
  always_rerank_positives=True,
  show_progress_bar=True
)

evaluator(model)

{'amh-passage-retrieval-dev_map': 0.8267625270131399,
 'amh-passage-retrieval-dev_mrr@10': 0.8246759530314858,
 'amh-passage-retrieval-dev_ndcg@10': 0.8578685327553554}

In [ ]:
from sentence_transformers.cross_encoder import CrossEncoder

# model = CrossEncoder("rasyosef/roberta-amharic-reranker-medium")
scores = model.predict([
    ["በለንደን የዩክሬን ደጋፊ አገራት መሪዎች ጉባኤ", "ጀርመን አገር የተማሩ ኢትዮጵያዉያን ቁጥር ከአፍሪቃ አንደኛ ነዉ። እነ ካሜሩን ከነኬንያን ሁሉ ይበልጣል። በማኅበር የመደራጀታችን ዋናዉ ዓላማ የኢትዮ-ጀርመንን ግንኙነትን ከፍ ማድረግ ነዉ። ከ 10 ሺህ በላይ ኢትዮጵያዉያን ተማሪዎች በጀርመን አገር ተምረዉ ተመልሰዋል። አልያም አዉሮጳ ወደ አሜሪካም የተሻገሩ አሉ።"],
    ["በለንደን የዩክሬን ደጋፊ አገራት መሪዎች ጉባኤ", "በጋምቤላ እስካሁን 23 ሰዎች በኮሌራ መሞታቸውን የተናገሩት አንድ የጤና ባለሙያ በሽታውን ለመከላከል አንድ የግል ተቋም ድጋፍ ማድረጉን እና በዓለም ጤና ድርጅት የህክምና ድጋፍ መሰጠቱን አብራርተዋል፡፡ ከደቡብ ሱዳን አጎራባች አኮቦ ወረዳ ለመጀመሪያ ጊዜ መከሰቱ በተነገረው nበሽታው ብዙ ሰዎች የተጠቁት ዋንቱዋ በተባለ ወረዳ መሆኑን ጠቁመዋል፡፡"],
    ["በለንደን የዩክሬን ደጋፊ አገራት መሪዎች ጉባኤ", "በደም እጥረት ምክንያት ለህሙማን አስፈላጊዉን ህክምና ለመስጠት ተቸግረናል ሲሉ የተናገሩት የቆቦ ሆስፒታል ስራ አስኪያጅ ቢኖሩም የደም እጥረቱ ሀኪሞች አገልግሎት እንዲሰጡ አላስቻላቸዉም ይላሉ። የወልድያ ደም ባንክ አገልግሎት ስራ አስኪያጅ አሁን ለአስቸኳይ ህክምና ብቻ የሚሆን ለ20 ቀናት ብቻ የሚቆይ ደም ነዉ ያለን ይላሉ"],
    ["በለንደን የዩክሬን ደጋፊ አገራት መሪዎች ጉባኤ", "አሜሪካ ለውጭ የምትሰጠውን እርዳታ ማቆሟ በተለይ በአፍሪቃ ሃገራት ከፍተኛ ግርታና ድንጋጤ ፈጥሯል። ለሁለተኛ ጊዜ ወደ የሥልጣን መንበር የመጡት የአሜሪካው ፕሬዝደንት ዶናልድ ትራምፕ የውጭ እርዳታ ለዘጠና ቀናት እንዲቆም ወስነዋል። ትራምፕ እርዳታ አፍሪቃውያንን አሳንፏቸዋል ነው የሚሉት።"],
    ["በለንደን የዩክሬን ደጋፊ አገራት መሪዎች ጉባኤ", "በለንደን የብሪታኒያው ጠ/ሚኒ ኬይር ስታርመር አስተናጋጅነት የተካሄደው ዋናዎቹ የዩክሬን ደጋፊ አገሮች መሪዎች ጉባኤ ዩክሬንን በወታደራዊና ፋይናናስ የበለጠ ለመርዳት ተስማምቶ ማብቃቱ ተገልጿል። ይህ ከአውሮፓ ህብረት ውጭ የካናዳንና የኖርዌይ መሪዎች ብሎም የቱርክን የውጭ ጉዳይ ሚኒስትር ጉባኤ የተጠራዉ የነጬ ቤተመንግስት ክስተትን ተከትሎ ነዉ።"]
  ])
scores

array([3.4760369e-04, 3.4751720e-04, 9.3876332e-04, 4.1233754e-04,
       9.9968314e-01], dtype=float32)

In [ ]:
from google.colab import userdata

# model.push_to_hub("roberta-amharic-reranker-medium", exist_ok=True, token=userdata.get("HF_WRITE"))